In [1]:
# %load /Users/hwayment/ipynb_defaults.py
%load_ext autoreload
%autoreload 2

%pylab inline
import os, sys
import json

import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('ticks')
sns.set_context('paper')

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score

Populating the interactive namespace from numpy and matplotlib


In [33]:
sys.path.extend(['/mnt/c/Users/abreha.b/OneDrive - Stanford/rotations/das/arnie', '/mnt/c/Users/abreha.b/OneDrive - Stanford/rotations/das/'])
os.environ["ARNIEFILE"] = '/mnt/c/Users/abreha.b/OneDrive - Stanford/rotations/das/arnie/run.arnie'

from DegScore import assign_loop_type
from assign_loop_type import write_loop_assignments

sys.path.append('/mnt/c/Users/abreha.b/OneDrive - Stanford/rotations/das/DegScore')

from DegScore import DegScore
from arnie.sample_structures import sample_structures

from tqdm.notebook import tqdm

## Code to reproduce DegScore from Kaggle datasets

In [14]:
def get_ensemble(sequence, n=1000):
    structs = sample_structures(sequence, n_samples=n, package='eternafold')
    structs = [list(write_loop_assignments(s)) for s in structs]
    return np.array(structs)

def encode_input(df, window_size=1, pad=10, seq=True, struct=True, ensemble_size=0):
    '''Creat input/output for regression model for predicting structure probing data.
    Inputs:
    
    dataframe (in EternaBench RDAT format)
    window_size: size of window (in one direction). so window_size=1 is a total window size of 3
    pad: number of nucleotides at start to not include
    seq (bool): include sequence encoding
    struct (bool): include bpRNA structure encoding
    
    Outputs:
    Input array (n_samples x n_features): array of windowed input features
    feature_names (list): feature names
    
    '''
    MAX_LEN = 68
    BASES = ['A','U','G','C']
    STRUCTS = ['H','E','I','M','B','S']
    
    inpts = []
    labels = []

    feature_kernel=[]
    if seq:
        feature_kernel.extend(BASES)
    if struct:
        feature_kernel.extend(STRUCTS)

    feature_names = ['%s_%d' % (k, val) for val in range(-1*window_size, window_size+1) for k in feature_kernel]
    
    for i, row in tqdm(df.iterrows(), desc='Encoding inputs', total=len(df)):
        arr = np.zeros([MAX_LEN,len(feature_kernel)])
        
        if ensemble_size > 0: # stochastically sample ensemble
            ensemble = get_ensemble(row['sequence'], n=ensemble_size)
        else: # use MEA structure
            ensemble = np.array([list(row['predicted_loop_type'])])

        for index in range(pad,MAX_LEN):
            ctr=0

            #encode sequence
            if seq:
                for char in BASES:
                    if row['sequence'][index]==char:
                        arr[index,ctr]+=1
                    ctr+=1

            if struct:
                loop_assignments = ''.join(ensemble[:,index])
                for char in STRUCTS:
                    prob = loop_assignments.count(char) / len(loop_assignments)
                    arr[index,ctr]+=prob
                    ctr+=1
                    
        # add zero padding to the side
        padded_arr = np.vstack([np.zeros([window_size,len(feature_kernel)]),arr[pad:], np.zeros([window_size,len(feature_kernel)])])

        for index in range(pad,MAX_LEN):
            new_index = index+window_size-pad
            tmp = padded_arr[new_index-window_size:new_index+window_size+1]
            inpts.append(tmp.flatten())
            labels.append('%s_%d' % (row['id'], index))
            
    return np.array(inpts), feature_names, labels

def encode_output(df, data_type='reactivity', pad=10):
    '''Creat input/output for regression model for predicting structure probing data.
    Inputs:
    
    dataframe (in EternaBench RDAT format)
    data_type: column name for degradation
    window_size: size of window (in one direction). so window_size=1 is a total window size of 3
    pad: number of nucleotides at start to not include
    
    Outputs:
    output array (n_samples): array of reactivity values
    
    '''
    MAX_LEN = 68
    
    outpts = []
    labels = []
    # output identity should be in form id_00073f8be_0

    for i, row in df.iterrows():
        for index in range(pad,MAX_LEN):
            outpts.append(row[data_type][index])
            labels.append('%s_%d' % (row['id'], index))
            
    return outpts, labels

#### Load data 

In [15]:
kaggle_train = pd.read_json('train.json',lines=True)
kaggle_train = kaggle_train.loc[kaggle_train['SN_filter']==1]

#kaggle_test = pd.read_json('test.json',lines=True)
kaggle_test = pd.read_csv('test_labels.csv')

#### Encode data

###### Max. expected accuracy

In [16]:
mea_inputs_train, mea_feature_names, _ = encode_input(kaggle_train, window_size=12)
mea_inputs_test, _, mea_test_labels = encode_input(kaggle_test, window_size=12)

###### Ensemble-averaged encoding

In [17]:
ensemble_size = 100
ens_inputs_train, ens_feature_names, _ = encode_input(kaggle_train, window_size=12, ensemble_size=ensemble_size)
ens_inputs_test, _, ens_test_labels = encode_input(kaggle_test, window_size=12, ensemble_size=ensemble_size)

#### Train models

###### MEA models

In [22]:
mea_models = {}

for output_type in ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C','deg_50C']:
    mea_outputs_train, mea_outputs_labels = encode_output(kaggle_train, data_type=output_type)

    # Clip negative values to 0
    mea_outputs_train = np.clip(mea_outputs_train, 0, 100)

    reg = Ridge(alpha=0.15)
    print('Fitting %s ...' % output_type)
    reg.fit(mea_inputs_train, mea_outputs_train)
    
    mea_models[output_type] = reg

Fitting reactivity ...
Fitting deg_Mg_pH10 ...
Fitting deg_pH10 ...
Fitting deg_Mg_50C ...
Fitting deg_50C ...


###### Ensemble-averaged models

In [23]:
ens_models = {}

for output_type in ['reactivity', 'deg_Mg_pH10', 'deg_pH10', 'deg_Mg_50C','deg_50C']:
    ens_outputs_train, ens_outputs_labels = encode_output(kaggle_train, data_type=output_type)
    
    # Clip negative values to 0
    ens_outputs_train = np.clip(ens_outputs_train, 0, 100)

    reg = Ridge(alpha=0.15)
    print('Fitting %s ...' % output_type)
    reg.fit(ens_inputs_train, ens_outputs_train)
    
    ens_models[output_type] = reg

Fitting reactivity ...
Fitting deg_Mg_pH10 ...
Fitting deg_pH10 ...
Fitting deg_Mg_50C ...
Fitting deg_50C ...


## Using the existing class that contains "DegScore-2.1"

In [30]:
def score_mcrmse(models, test_df):
    scored_data_types = ['reactivity', 'deg_Mg_pH10', 'deg_Mg_50C']

    seq_scores = []
    for i, row in test_df.iterrows():
        scores = []
        for data_type in scored_data_types:
            model = models[data_type]
            
            coeffs = model.coef_
            inter = model.intercept_
            
            mdl = DegScore.DegScore(row['sequence'], structure=row['structure'], coeffs=coeffs, intercept=inter)
            
            seq_scored = row['seq_scored']
            pred = mdl.degscore_by_position[:seq_scored]
            target = np.array(json.loads(row[data_type])[:seq_scored])
            
            scores.extend(np.square(pred - target))
        
        seq_scores.append(scores)
        
    return np.mean(seq_scores)

In [31]:
score_mcrmse(mea_models, kaggle_test)

0.24691545925204095

In [32]:
score_mcrmse(ens_models, kaggle_test)

0.23856658661289426